# Data Acquisition and Exploration

This notebook downloads and explores the Multi-Turn Insurance Underwriting dataset from Hugging Face.

**Dataset**: `snorkelai/Multi-Turn-Insurance-Underwriting`

**Objectives**:
1. Download dataset from Hugging Face
2. Understand data schema and structure
3. Analyze distribution of examples
4. Identify data quality issues
5. Document findings

In [ ]:
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset
from collections import Counter
import json

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

## 1. Load Dataset

In [ ]:
# Load dataset from Hugging Face
print("Loading dataset from Hugging Face...")
dataset = load_dataset("snorkelai/Multi-Turn-Insurance-Underwriting")

print(f"\nDataset structure: {dataset}")
print(f"\nNumber of examples: {len(dataset['train'])}")

In [ ]:
# Examine first example
example = dataset['train'][0]
print("First example keys:")
print(json.dumps({k: type(v).__name__ for k, v in example.items()}, indent=2))

print("\nFirst example:")
print(json.dumps(example, indent=2, default=str)[:2000] + "...")

## 2. Schema Analysis

In [ ]:
# Analyze schema
print("Dataset features:")
print(dataset['train'].features)

# Check column names
print("\nColumn names:")
print(dataset['train'].column_names)

## 3. Data Distribution Analysis

In [ ]:
# Convert to pandas for easier analysis
df = dataset['train'].to_pandas()
print(f"Total examples: {len(df)}")
print(f"\nDataFrame shape: {df.shape}")
print(f"\nDataFrame columns: {df.columns.tolist()}")

In [ ]:
# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

print("\nMissing value percentages:")
print((df.isnull().sum() / len(df) * 100).round(2))

## 4. Conversation Structure Analysis

In [ ]:
# Analyze conversation structure
# Note: The actual structure depends on the dataset format
# This is a placeholder that will be updated based on actual data

def count_conversation_turns(example):
    """Count number of turns in a conversation."""
    # Placeholder - will be updated based on actual data structure
    if 'messages' in example:
        return len(example['messages'])
    elif 'conversation' in example:
        return len(example['conversation'])
    else:
        return 0

# Apply to dataset
conversation_lengths = [count_conversation_turns(ex) for ex in dataset['train']]

print(f"Conversation length statistics:")
print(f"  Mean: {np.mean(conversation_lengths):.2f}")
print(f"  Median: {np.median(conversation_lengths):.2f}")
print(f"  Min: {np.min(conversation_lengths)}")
print(f"  Max: {np.max(conversation_lengths)}")
print(f"  Std: {np.std(conversation_lengths):.2f}")

In [ ]:
# Visualize conversation length distribution
plt.figure(figsize=(10, 6))
plt.hist(conversation_lengths, bins=20, edgecolor='black')
plt.xlabel('Number of Turns')
plt.ylabel('Frequency')
plt.title('Distribution of Conversation Lengths')
plt.grid(axis='y', alpha=0.3)
plt.show()

## 5. Company Profile Analysis

In [ ]:
# Analyze company profiles
# Placeholder - will be updated based on actual data structure

def extract_company_info(example):
    """Extract company information from example."""
    # Placeholder - will be updated based on actual structure
    return {
        'revenue': example.get('annual_revenue'),
        'employees': example.get('number_of_employees'),
        'industry': example.get('industry'),
        'state': example.get('state'),
    }

# Extract company information
company_info = [extract_company_info(ex) for ex in dataset['train']]
company_df = pd.DataFrame(company_info)

print("Company profile fields:")
print(company_df.head())

## 6. Task Type Distribution

In [ ]:
# Analyze task types
# Placeholder - will be updated based on actual data

def identify_task_type(example):
    """Identify the task type of an example."""
    # Placeholder - will be updated based on actual structure
    # Common task types: appetite check, product recommendation, eligibility, etc.
    return example.get('task_type', 'unknown')

task_types = [identify_task_type(ex) for ex in dataset['train']]
task_distribution = Counter(task_types)

print("Task type distribution:")
for task, count in task_distribution.most_common():
    print(f"  {task}: {count} ({count/len(task_types)*100:.1f}%)")

In [ ]:
# Visualize task distribution
plt.figure(figsize=(10, 6))
tasks, counts = zip(*task_distribution.most_common())
plt.bar(tasks, counts)
plt.xlabel('Task Type')
plt.ylabel('Count')
plt.title('Distribution of Task Types')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## 7. Text Length Analysis

In [ ]:
# Analyze text lengths (approximate token counts)
def estimate_tokens(text):
    """Estimate token count (rough approximation: words / 0.75)."""
    return int(len(text.split()) / 0.75)

def get_total_text_length(example):
    """Get total text length for an example."""
    # Placeholder - will be updated based on actual structure
    total = 0
    if 'messages' in example:
        for msg in example['messages']:
            if isinstance(msg, dict) and 'content' in msg:
                total += estimate_tokens(msg['content'])
    return total

text_lengths = [get_total_text_length(ex) for ex in dataset['train']]

print(f"Text length statistics (tokens):")
print(f"  Mean: {np.mean(text_lengths):.0f}")
print(f"  Median: {np.median(text_lengths):.0f}")
print(f"  Min: {np.min(text_lengths)}")
print(f"  Max: {np.max(text_lengths)}")
print(f"  Std: {np.std(text_lengths):.0f}")

## 8. Data Quality Issues

In [ ]:
# Identify data quality issues
issues = []

for idx, example in enumerate(dataset['train']):
    # Check for tool calls (to be excluded)
    example_str = str(example).lower()
    if 'tool_call' in example_str or 'function_call' in example_str:
        issues.append({
            'index': idx,
            'issue': 'Contains tool/function calls',
        })
    
    # Check for missing company profile
    if not any(field in example for field in ['company', 'business', 'annual_revenue']):
        issues.append({
            'index': idx,
            'issue': 'Missing company profile',
        })
    
    # Check for empty conversations
    if count_conversation_turns(example) == 0:
        issues.append({
            'index': idx,
            'issue': 'Empty conversation',
        })

print(f"Total data quality issues found: {len(issues)}")
if issues:
    issues_df = pd.DataFrame(issues)
    print("\nIssue distribution:")
    print(issues_df['issue'].value_counts())

## 9. Sample Examples

In [ ]:
# Display a few sample examples
print("Sample Example 1:")
print("=" * 80)
print(json.dumps(dataset['train'][0], indent=2, default=str)[:1500])
print("\n...\n")

In [ ]:
print("Sample Example 2:")
print("=" * 80)
print(json.dumps(dataset['train'][1], indent=2, default=str)[:1500])
print("\n...\n")

## 10. Summary and Findings

### Key Findings:

1. **Dataset Size**: [TO BE FILLED AFTER RUNNING]
2. **Schema**: [TO BE FILLED AFTER RUNNING]
3. **Conversation Structure**: [TO BE FILLED AFTER RUNNING]
4. **Task Distribution**: [TO BE FILLED AFTER RUNNING]
5. **Data Quality**: [TO BE FILLED AFTER RUNNING]

### Next Steps:

1. Implement preprocessing pipeline based on schema
2. Handle data quality issues (filter tool calls, etc.)
3. Create train/validation/test splits
4. Implement tokenization for chosen model


In [ ]:
# Save dataset locally for faster access
output_dir = project_root / "data" / "raw"
output_dir.mkdir(parents=True, exist_ok=True)

dataset.save_to_disk(str(output_dir / "insurance_underwriting"))
print(f"Dataset saved to: {output_dir / 'insurance_underwriting'}")

In [ ]:
# Save exploration summary
summary = {
    'total_examples': len(dataset['train']),
    'conversation_length_stats': {
        'mean': float(np.mean(conversation_lengths)),
        'median': float(np.median(conversation_lengths)),
        'min': int(np.min(conversation_lengths)),
        'max': int(np.max(conversation_lengths)),
    },
    'text_length_stats': {
        'mean': float(np.mean(text_lengths)),
        'median': float(np.median(text_lengths)),
        'min': int(np.min(text_lengths)),
        'max': int(np.max(text_lengths)),
    },
    'task_distribution': dict(task_distribution),
    'quality_issues': len(issues),
}

summary_path = project_root / "data" / "exploration_summary.json"
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"Summary saved to: {summary_path}")